#Licenciamiento de universidades peruanas

##Importar librerías

In [202]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

##Carga de datos

In [203]:
data=pd.read_excel('dataset.xlsx')
data.head()

,CODIGO_ENTIDAD,NOMBRE,TIPO_GESTION,ESTADO_LICENCIAMIENTO,FECHA_INICIO_LICENCIAMIENTO,FECHA_FIN_LICENCIAMIENTO,PERIODO_LICENCIAMIENTO,DEPARTAMENTO,PROVINCIA,DISTRITO,UBIGEO,LATITUD,LONGITUD,FECHA_CORTE
0,1,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,PÚBLICO,LICENCIA OTORGADA,2018-04-10 00:00:00,2028-04-10,10,LIMA,LIMA,LIMA,140101,-12.056026,-77.084423,2022-08-17
1,2,UNIVERSIDAD NACIONAL DE SAN CRISTÓBAL DE HUAMANGA,PÚBLICO,LICENCIA OTORGADA,2018-06-01 00:00:00,2024-06-01,6,AYACUCHO,HUAMANGA,AYACUCHO,50101,-13.161265,-74.225734,2022-08-17
2,4,UNIVERSIDAD NACIONAL DE TRUJILLO,PÚBLICO,LICENCIA OTORGADA,2018-09-21 00:00:00,2026-09-21,8,LA LIBERTAD,TRUJILLO,TRUJILLO,120101,-8.114378,-79.037901,2022-08-17
3,5,UNIVERSIDAD NACIONAL DE SAN AGUSTÍN DE AREQUIPA,PÚBLICO,LICENCIA OTORGADA,2018-08-27 00:00:00,2028-08-27,10,AREQUIPA,AREQUIPA,AREQUIPA,40101,-16.403293,-71.524889,2022-08-17
4,6,UNIVERSIDAD NACIONAL DE INGENIERÍA,PÚBLICO,LICENCIA OTORGADA,2017-11-28 00:00:00,2027-11-28,10,LIMA,LIMA,RÍMAC,140122,-12.023739,-77.048376,2022-08-17


##EDA

In [204]:
data.shape

(143, 14)

In [205]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   CODIGO_ENTIDAD               143 non-null    int64         
 1   NOMBRE                       143 non-null    object        
 2   TIPO_GESTION                 143 non-null    object        
 3   ESTADO_LICENCIAMIENTO        143 non-null    object        
 4   FECHA_INICIO_LICENCIAMIENTO  93 non-null     object        
 5   FECHA_FIN_LICENCIAMIENTO     93 non-null     datetime64[ns]
 6   PERIODO_LICENCIAMIENTO       143 non-null    int64         
 7   DEPARTAMENTO                 143 non-null    object        
 8   PROVINCIA                    143 non-null    object        
 9   DISTRITO                     143 non-null    object        
 10  UBIGEO                       143 non-null    int64         
 11  LATITUD                      143 non-null    

###Localización de las universidades analizadas

In [206]:
import folium
# Creamos el mapa centrado en Buenos Aires
mapa = folium.Map(location=[-12.046374, -77.042793], zoom_start=6)

# Agregamos los puntos de las localidades
for index, row in data.iterrows():
    folium.Marker([row['LATITUD'], row['LONGITUD']], popup=row['NOMBRE']).add_to(mapa)

# Agregamos las burbujas con el tamaño de la población
for index, row in data.iterrows():
    folium.Circle(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=row[['NOMBRE', 'DEPARTAMENTO']],  # agregamos localidad y poblacion para que aparezca como detalle al hacer click en alguna de las burbujas
        fill=True,
        fill_opacity=0.7,
        color='red'
    ).add_to(mapa)

# Agregamos una capa de control al mapa
folium.LayerControl().add_to(mapa)

mapa

###Estado de licenciamiento de las universidades

In [207]:
est_lic=data.value_counts('ESTADO_LICENCIAMIENTO')
est_lic

ESTADO_LICENCIAMIENTO
LICENCIA OTORGADA                               93
LICENCIA DENEGADA                               48
CON INFORME DE OBSERVACIONES (IO) NOTIFICADO     1
NINGUNO                                          1
dtype: int64

In [208]:
est_lic=pd.DataFrame(est_lic,columns=['cantidad'])
est_lic.reset_index(inplace=True)
est_lic

,ESTADO_LICENCIAMIENTO,cantidad
0,LICENCIA OTORGADA,93
1,LICENCIA DENEGADA,48
2,CON INFORME DE OBSERVACIONES (IO) NOTIFICADO,1
3,NINGUNO,1


In [210]:
est_gest=data.groupby(['ESTADO_LICENCIAMIENTO','TIPO_GESTION'])['TIPO_GESTION'].count()
est_gest

ESTADO_LICENCIAMIENTO                         TIPO_GESTION
CON INFORME DE OBSERVACIONES (IO) NOTIFICADO  PÚBLICO          1
LICENCIA DENEGADA                             PRIVADO         46
                                              PÚBLICO          2
LICENCIA OTORGADA                             PRIVADO         46
                                              PÚBLICO         47
NINGUNO                                       PÚBLICO          1
Name: TIPO_GESTION, dtype: int64

In [209]:
fig = px.histogram(est_lic, x='ESTADO_LICENCIAMIENTO', y='cantidad', text_auto = True)
fig.update_traces(textfont_size = 14, textangle = 0, textposition = "outside")
fig.show()

###Departamentos y provincias con más licencias otorgadas

In [211]:
est_dep=data[data['ESTADO_LICENCIAMIENTO']=='LICENCIA OTORGADA'].groupby(['DEPARTAMENTO'])['ESTADO_LICENCIAMIENTO'].count()
est_dep.sort_values(ascending=False).head(10)

DEPARTAMENTO
LIMA            36
JUNÍN            6
LA LIBERTAD      5
AREQUIPA         4
PIURA            3
CAJAMARCA        3
CUSCO            3
HUANCAVELICA     3
HUÁNUCO          3
APURÍMAC         3
Name: ESTADO_LICENCIAMIENTO, dtype: int64

In [212]:
est_dep_dist=data[data['ESTADO_LICENCIAMIENTO']=='LICENCIA OTORGADA'].groupby(['DEPARTAMENTO','DISTRITO'])['ESTADO_LICENCIAMIENTO'].count()
est_dep_dist.sort_values(ascending=False).head(10)

DEPARTAMENTO  DISTRITO         
LIMA          SANTIAGO DE SURCO    5
              LA MOLINA            4
              LIMA                 4
AREQUIPA      AREQUIPA             4
LA LIBERTAD   TRUJILLO             3
JUNÍN         HUANCAYO             3
LIMA          VILLA EL SALVADOR    3
              JESÚS MARÍA          2
              LOS OLIVOS           2
              LURIGANCHO           2
Name: ESTADO_LICENCIAMIENTO, dtype: int64

###Universidades próximas al vencimiento de su licencia

In [213]:
import datetime
fecha_actual = pd.to_datetime(datetime.date.today())
fin_lic =data[data['FECHA_FIN_LICENCIAMIENTO'] >= fecha_actual]
universidades_fin_lic=fin_lic[['NOMBRE','DEPARTAMENTO','PROVINCIA','DISTRITO','FECHA_FIN_LICENCIAMIENTO']].sort_values(by=['FECHA_FIN_LICENCIAMIENTO'],ascending=True).head(10)
universidades_fin_lic

,NOMBRE,DEPARTAMENTO,PROVINCIA,DISTRITO,FECHA_FIN_LICENCIAMIENTO
14,UNIVERSIDAD DE SAN MARTÍN DE PORRES,LIMA,LIMA,SANTA ANITA,2023-08-10
33,UNIVERSIDAD NACIONAL TORIBIO RODRÍGUEZ DE MEND...,AMAZONAS,CHACHAPOYAS,CHACHAPOYAS,2023-09-27
38,UNIVERSIDAD NACIONAL JOSÉ MARÍA ARGUEDAS,APURÍMAC,ANDAHUAYLAS,SAN JERÓNIMO,2023-10-03
26,UNIVERSIDAD PERUANA DE CIENCIAS APLICADAS S.A.C.,LIMA,LIMA,SANTIAGO DE SURCO,2023-10-05
28,UNIVERSIDAD SAN IGNACIO DE LOYOLA S.A.,LIMA,LIMA,LA MOLINA,2023-10-09
25,UNIVERSIDAD MARCELINO CHAMPAGNAT,LIMA,LIMA,SANTIAGO DE SURCO,2023-10-11
29,UNIVERSIDAD CATÓLICA SAN PABLO,AREQUIPA,AREQUIPA,AREQUIPA,2023-10-21
27,UNIVERSIDAD PRIVADA DEL NORTE S.A.C.,LA LIBERTAD,TRUJILLO,TRUJILLO,2023-11-13
41,ASOCIACIÓN CIVIL UNIVERSIDAD DE CIENCIAS Y HUM...,LIMA,LIMA,LOS OLIVOS,2023-11-20
23,UNIVERSIDAD PRIVADA DE TACNA,TACNA,TACNA,POCOLLAY,2023-11-30
